# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [170]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint
# Import API key
from api_keys import gkey


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [154]:
file = ("clean_output_data.csv")
df = pd.read_csv(file)
df=df.reset_index()
df=df.drop(['index'],axis=1)
df

,Unnamed: 0,city,Country,Date,Latitude,Longitude,Cloudiness,Humidity,Max_Temp,Wind_Speed,Temperature
0,0,punta arenas,CL,1615977347,-53.1500,-70.9167,90,71,283.15,10.29,50.270
1,1,bluff,NZ,1615977041,-46.6000,168.3333,22,80,281.48,0.45,47.264
2,2,chernyshevskiy,RU,1615977595,63.0128,112.4714,98,95,254.93,2.46,-0.526
3,3,hobart,AU,1615977595,-42.8794,147.3294,0,67,289.15,1.03,61.070
4,4,bow island,CA,1615977596,49.8668,-111.3684,0,74,269.15,0.51,25.070
...,...,...,...,...,...,...,...,...,...,...,...
539,556,stephenville,US,1615977997,32.2207,-98.2023,40,94,287.59,1.54,58.262
540,557,koryazhma,RU,1615977998,61.3143,47.1691,100,94,273.71,2.54,33.278
541,558,waingapu,ID,1615977999,-9.6567,120.2641,100,84,299.94,2.46,80.492
542,559,dashitou,CN,1615978000,43.3067,128.5114,0,58,270.60,3.52,27.680


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [155]:
gmaps.configure(api_key=gkey)
coordinates = df[["Latitude", "Longitude"]]
humidity = df["Humidity"]
fig = gmaps.figure(center = (40.712743, -74.013379), zoom_level = 1.5)

heat_layer = gmaps.heatmap_layer(coordinates, weights=humidity, dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [156]:
new_df = df.loc[df["Temperature"] > 70] 
new_df_01 = new_df.loc[new_df["Temperature"] < 80]
new_df_02 = new_df_01.loc[new_df_01["Wind_Speed"]<10]
new_df_03 = new_df_02.loc[new_df_02["Cloudiness"] ==0]

hotel_df = new_df_03.reset_index()
del hotel_df["index"]
del hotel_df["Unnamed: 0"]
hotel_df


,city,Country,Date,Latitude,Longitude,Cloudiness,Humidity,Max_Temp,Wind_Speed,Temperature
0,sao filipe,CV,1615977118,14.8961,-24.4956,0,74,295.19,2.07,71.942
1,victoria,HK,1615977053,22.2855,114.1577,0,69,298.71,4.92,78.278
2,cape town,ZA,1615977416,-33.9258,18.4232,0,50,299.26,4.12,79.268
3,geraldton,AU,1615977193,-28.7667,114.6000,0,73,298.15,9.77,77.270
4,sao joao da barra,BR,1615977675,-21.6403,-41.0511,0,88,297.15,5.24,75.470
5,luderitz,NaN,1615977784,-26.6481,15.1594,0,62,295.19,6.22,71.942
6,port augusta,AU,1615977785,-32.5000,137.7667,0,73,295.37,5.14,72.266
7,springbok,ZA,1615977840,-29.6643,17.8865,0,31,299.29,5.19,79.322
8,barao de melgaco,BR,1615977867,-16.1944,-55.9675,0,94,297.15,1.03,75.470
9,teahupoo,PF,1615977870,-17.8333,-149.2667,0,73,299.15,1.03,79.070


In [157]:
hotel_df["Hotel_Name"] = ""
hotel_df

,city,Country,Date,Latitude,Longitude,Cloudiness,Humidity,Max_Temp,Wind_Speed,Temperature,Hotel_Name
0,sao filipe,CV,1615977118,14.8961,-24.4956,0,74,295.19,2.07,71.942,
1,victoria,HK,1615977053,22.2855,114.1577,0,69,298.71,4.92,78.278,
2,cape town,ZA,1615977416,-33.9258,18.4232,0,50,299.26,4.12,79.268,
3,geraldton,AU,1615977193,-28.7667,114.6000,0,73,298.15,9.77,77.270,
4,sao joao da barra,BR,1615977675,-21.6403,-41.0511,0,88,297.15,5.24,75.470,
5,luderitz,NaN,1615977784,-26.6481,15.1594,0,62,295.19,6.22,71.942,
6,port augusta,AU,1615977785,-32.5000,137.7667,0,73,295.37,5.14,72.266,
7,springbok,ZA,1615977840,-29.6643,17.8865,0,31,299.29,5.19,79.322,
8,barao de melgaco,BR,1615977867,-16.1944,-55.9675,0,94,297.15,1.03,75.470,
9,teahupoo,PF,1615977870,-17.8333,-149.2667,0,73,299.15,1.03,79.070,


# Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [171]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
params = {"key": gkey,
         "radius": 5000,
         "keyword": "Hotel"}


In [162]:
for index, row in hotel_df.iterrows():
    
    lat = row["Latitude"]
    lng = row["Longitude"]
    
    params["location"] = f"{lat},{lng}"
    response = requests.get(base_url, params=params).json()
   
    
    try:
        hotel_df.loc[index, "Hotel_Name"] = response["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing")
        

Missing
Missing


In [163]:
hotel_df

,city,Country,Date,Latitude,Longitude,Cloudiness,Humidity,Max_Temp,Wind_Speed,Temperature,Hotel_Name
0,sao filipe,CV,1615977118,14.8961,-24.4956,0,74,295.19,2.07,71.942,La Fora Ecolodge
1,victoria,HK,1615977053,22.2855,114.1577,0,69,298.71,4.92,78.278,Ovolo Central
2,cape town,ZA,1615977416,-33.9258,18.4232,0,50,299.26,4.12,79.268,Atlanticview Cape Town Boutique Hotel
3,geraldton,AU,1615977193,-28.7667,114.6000,0,73,298.15,9.77,77.270,Ocean Centre Hotel
4,sao joao da barra,BR,1615977675,-21.6403,-41.0511,0,88,297.15,5.24,75.470,Pousada Kactus
5,luderitz,NaN,1615977784,-26.6481,15.1594,0,62,295.19,6.22,71.942,Prospectors Inn Hotel
6,port augusta,AU,1615977785,-32.5000,137.7667,0,73,295.37,5.14,72.266,Majestic Oasis Apartments
7,springbok,ZA,1615977840,-29.6643,17.8865,0,31,299.29,5.19,79.322,Springbok Hotel
8,barao de melgaco,BR,1615977867,-16.1944,-55.9675,0,94,297.15,1.03,75.470,Pousada Barão de Melgaço
9,teahupoo,PF,1615977870,-17.8333,-149.2667,0,73,299.15,1.03,79.070,Vanira Lodge


In [166]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel_Name}</dd>
<dt>city</dt><dd>{city}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [169]:
# Add marker layer ontop of heat map


marker_layer = gmaps.marker_layer(locations)


# Add layer
fig.add_layer(marker_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))